In [1]:
import pandas as pd
import numpy as np
import openpyxl
import os
from datetime import datetime


In [ ]:
pathX = os.getcwd()+"\data\X"
pathY = os.getcwd()+"\data\Y"
file_list_all = os.listdir(pathY)

In [ ]:
# 파일명 및 파일 확장자 지정
file_list = []

for file in file_list_all:
    if file.startswith("연립다세대(매매)") & file.endswith(".xlsx"):
        print(file)
        file_list.append(file)

In [ ]:
df = pd.DataFrame()

for file in file_list:
    print(file)
    data = pd.read_excel(pathY+"\\"+file, header=0, skiprows=16)
    df = pd.concat([df,data])

In [ ]:
# row 수 확인
print('row count = {}'.format(len(df)))

# data 확인
print(df.head(5))

In [ ]:
# dataframe column data type 확인
#df.dtypes

# dataframe 대략적인 정보 확인
#df.info()

# 건축년도 nan 데이터 처리(서울특별시 성북구 길음동 1083) -> 0 으로 치환, 데이터 타입 변환 int
#df[df.건축년도.isna()]

df['건축년도'] = df['건축년도'].fillna(0).astype(int)

print(df[(df.번지=='1083')])

In [30]:
# '거래금액(만원)' 쉼표 제거 및 데이터 타입 변환(numeric)
# '거래금액(만원)' 단위 만원 -> 원 으로 변경
#df['거래금액(만원)']
df['거래금액(만원)'] = pd.to_numeric(df['거래금액(만원)'].apply(lambda x: x.replace(',', '')))
df['거래금액(만원)'] = df['거래금액(만원)'] * 10000

In [32]:
# 데이터 분리: 계약년월 -> 계약년도, 계약월
df['계약년월_str'] = df['계약년월'].apply(lambda x: str(x))
df['계약년도'] = df['계약년월_str'].apply(lambda x: x[:4])
df['계약월'] = df['계약년월_str'].apply(lambda x: x[4:])

In [33]:
# 계약일자: datetime 타입의 계약일
df['계약일자'] = df.계약년월*100 + df.계약일
df['계약일자'] = pd.to_datetime(df['계약일자'].apply(lambda x:str(x)))

In [34]:
# 시,구,동 컬럼 생성
df['시'] = df.시군구.str.split(' ').str[0]
df['구'] = df.시군구.str.split(' ').str[1]
df['동'] = df.시군구.str.split(' ').str[2]

In [35]:
# 도로명주소 분리: 도로명, 건물번호
df.rename(columns = {'도로명':'도로명주소'}, inplace=True)
df['도로명'] = df.도로명주소.str.split(' ').str[0]
df['건물번호'] = df.도로명주소.str.split(' ').str[1]

In [ ]:
df.columns

In [37]:
# 컬럼명 재정의
df.columns = ['시군구','번지','본번','부번','건물명','전용면적','대지권면적',
              '계약년월','계약일','거래금액','층',
              '건축년도','도로명주소','해제사유발생일',
              '계약년월_str','계약년도','계약월','계약일자',
             '시','구','동','도로명','건물번호']

In [45]:
# 인플레이션 반영한 거래금액 보정 by using GDP Deflator(기준년도:2015)
file_name = "GDP_Deflator_한국은행.xlsx"
deflator = pd.read_excel(pathX+"\\"+file_name, header=0)

In [46]:
# 행렬 전환
deflator = deflator.transpose()
deflator = deflator[4:]

# 컬럼명 재정의
deflator.reset_index(drop=False, inplace=True)
deflator.columns = ['년도','물가지수']

# 2021년 물가지수(예상치) append
deflator = deflator.append({'년도':'2021', '물가지수':106.5039}, ignore_index=True)

# 데이터 타입 변환(물가지수: float)
deflator['물가지수'] = deflator['물가지수'].apply(lambda x: float(x))
#deflator.info()

In [50]:
# 물가지수 left join
df = pd.merge(df, deflator, left_on='계약년도', right_on='년도', how='left')

In [ ]:
# 거래금액_물가반영
df['거래금액_물가반영'] = df.거래금액*(df.물가지수/100)
df['거래금액_물가반영'] = df['거래금액_물가반영'].astype(np.uint64)

# 거래금액_로그
df['거래금액_로그'] = df['거래금액_물가반영'].apply(lambda x: np.log(x))

In [ ]:
df['주소'] = df.시 + " " + df.구 + " " + df.동 + " " + df.도로명주소

In [71]:
thisyear = datetime.today().year                        # 현재 연도 가져오기
df['연식'] = thisyear  - pd.to_numeric(df.건축년도)     # 건물 연식

In [78]:
# 컬럼 순서 바꾸기
df = df[['주소', '시', '구', '동',
         '도로명주소','도로명','건물번호',
         '번지','본번','부번',
         '층','건물명','전용면적','대지권면적','건축년도','연식',
         '계약년월','계약년도','계약월','계약일','계약일자',
         '거래금액','거래금액_물가반영','거래금액_로그',
         '해제사유발생일']]

In [81]:
# 컬럼명 재정의
df.columns = ['addr', 'si', 'gu', 'dong',
            'addr_street','street_name','bldg_num',
            'beonji','bonbeon','bubeon',
            'floor','bldg_name','jeonyong_area','daejigwon_area','const_year','age',
            'contract_yyyymm','contract_yyyy','contract_mm','contract_day','contract_date',
            'cost','cost_reflected','cost_reflected_log',
            'cancel_date']

In [ ]:
# df.to_excel('merged_data_v2.xlsx')

print('*********************************************************************start to create excel file')
df.to_excel(pathY+"\\"+'merged_data_v2.xlsx', sheet_name='TRANSACTION_VILLA', index = False)
print('*********************************************************************End of merge_data.py')

***
***

In [92]:
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point

import googlemaps
import googlemap_key  # user 생성.py
gmaps = googlemaps.Client(key=googlemap_key.gmaps_key)

In [4]:
import pymssql

In [117]:
conn = pymssql.connect(host="####", user="####", password="####", database="####", charset="utf8")

In [118]:
sql = "select distinct addr, gu, dong from TRANSACTION_VILLA where gu=N'은평구' "

In [119]:
# DB의 데이터테이블을 dataframe으로 불러오기
# 중구 = pd.read_sql(sql=sql, con=conn)
# 성동구 = pd.read_sql(sql=sql, con=conn)
# 종로구 = pd.read_sql(sql=sql, con=conn)
# 영등포구 = pd.read_sql(sql=sql, con=conn)
# 노원구 = pd.read_sql(sql=sql, con=conn)
# 용산구 = pd.read_sql(sql=sql, con=conn)
# 동대문구 = pd.read_sql(sql=sql, con=conn)
# 강남구 = pd.read_sql(sql=sql, con=conn)
# 금천구 = pd.read_sql(sql=sql, con=conn)
# 서초구 = pd.read_sql(sql=sql, con=conn)
# 중랑구 = pd.read_sql(sql=sql, con=conn)
# 서대문구 = pd.read_sql(sql=sql, con=conn)
# 광진구 = pd.read_sql(sql=sql, con=conn)
# 도봉구 = pd.read_sql(sql=sql, con=conn)
# 동작구 = pd.read_sql(sql=sql, con=conn)
# 성북구 = pd.read_sql(sql=sql, con=conn)
# 마포구 = pd.read_sql(sql=sql, con=conn)
# 구로구 = pd.read_sql(sql=sql, con=conn)
# 강동구 = pd.read_sql(sql=sql, con=conn)
# 관악구 = pd.read_sql(sql=sql, con=conn)
# 강북구 = pd.read_sql(sql=sql, con=conn)
# 양천구 = pd.read_sql(sql=sql, con=conn)
# 송파구 = pd.read_sql(sql=sql, con=conn)
# 강서구 = pd.read_sql(sql=sql, con=conn)
은평구 = pd.read_sql(sql=sql, con=conn)

In [ ]:
# 중구.assign(lat='', lng='')
# 성동구.assign(lat='', lng='')
# 종로구.assign(lat='', lng='')
# 영등포구.assign(lat='', lng='')
# 노원구.assign(lat='', lng='')
# 용산구.assign(lat='', lng='')
# 동대문구.assign(lat='', lng='')
# 강남구.assign(lat='', lng='')
# 금천구.assign(lat='', lng='')
# 서초구.assign(lat='', lng='')
# 중랑구.assign(lat='', lng='')
# 서대문구.assign(lat='', lng='')
# 광진구.assign(lat='', lng='')
# 도봉구.assign(lat='', lng='')
# 동작구 = 동작구.assign(lat='', lng='')
# 성북구.assign(lat='', lng='')
# 마포구.assign(lat='', lng='')
# 구로구.assign(lat='', lng='')
# 강동구.assign(lat='', lng='')
# 관악구.assign(lat='', lng='')
# 강북구.assign(lat='', lng='')
# 양천구.assign(lat='', lng='')
# 송파구.assign(lat='', lng='')
# 강서구.assign(lat='', lng='')
은평구.assign(lat='', lng='')

In [ ]:
for idx, row in 은평구.iterrows():
    temp = gmaps.geocode(row.addr, language='ko')
    print(idx)
    
    if len(temp) == 0:  # 위/경도 return 못할 경우
        은평구.loc[idx,'lat'] = "NoData"
        은평구.loc[idx,'lng'] = "NoData"
        continue
    else:
        index = len(temp)-1
        은평구.loc[idx,'lat']=temp[index]['geometry']['location']['lat']
        은평구.loc[idx,'lng']=temp[index]['geometry']['location']['lng']

#for i, addr in enumerate(중구.addr):
#    list_addr = gmaps.geocode(addr, language='ko')
#    idx = len(list_addr) - 1
#    중구.loc[i,'lat'] = list_addr[idx]['geometry']['location']['lat']
#    중구.loc[i,'lng'] = list_addr[idx]['geometry']['location']['lng']

In [122]:
# 위경도 ret 방법1 - 엑셀파일
filename = "은평구.xlsx"
은평구.to_excel(pathX+"\\"+filename, sheet_name='LOCATION_EUNPYEONG', index=False, header=True)

In [ ]:
#성동구['lat'] = 성동구['lat'].astype(float)
#성동구['lng'] = 성동구['lng'].astype(float)

In [52]:
# Connection 으로부터 Cursor 생성
curs = conn.cursor() 

In [ ]:
# 위경도 ret 방법2 - DB Insert
sql = 'insert into LOCATION_GANGSEO values (%s, %s, %s, %s, %s)'
loc = 0

for i in 강서구.values:
    curs.execute(sql, tuple(i))
    print(loc)
    loc+=1

In [54]:
conn.commit()

In [123]:
# DB 연결 해제
conn.close()